<a href="https://colab.research.google.com/github/visumania/TFM-AMM/blob/main/cudernos/entrenaBertGenericoBueno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nlpaug

     |████████████████████████████████| 410 kB 5.1 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
import tensorflow as tf
import torch
import numpy as np
import imblearn
from imblearn.under_sampling import TomekLinks, EditedNearestNeighbours
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.under_sampling import ClusterCentroids
from collections import Counter
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SVMSMOTE
from keras.preprocessing.sequence import pad_sequences# Set the maximum sequence length.
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer, AutoModelForSequenceClassification, AutoTokenizer
#from transformers import * # Load the BERT tokenizer.
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import re

import nlpaug.augmenter.word as naw

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))

import pickle


     |████████████████████████████████| 3.5 MB 5.5 MB/s 
     |████████████████████████████████| 596 kB 34.8 MB/s 
     |████████████████████████████████| 67 kB 3.1 MB/s 
     |████████████████████████████████| 895 kB 4.8 MB/s 
     |████████████████████████████████| 6.8 MB 32.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
etiqueta = 'irony'

In [ ]:
MAX_LEN = 120

#df = pd.read_csv('/content/drive/MyDrive/I2C/trainEnFinal.csv', sep=',')
df = pd.read_csv('/content/drive/MyDrive/I2C/Adrián Moreno/Datasets/trainEnFinal.csv', sep=',')

df.drop('Unnamed: 0', inplace=True, axis=1)
del(df['rephrase'])
del(df['sarcastic'])

df = df.dropna()
df[['sarcasm','irony', 'satire', 'understatement', 'overstatement', 'rhetorical_question']]= df[['sarcasm','irony', 'satire', 'understatement', 'overstatement', 'rhetorical_question']].astype(int)

df

,tweet,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
0,The only thing I got from college is a caffein...,0,1,0,0,0,0
1,I love it when professors draw a big question ...,1,0,0,0,0,0
2,Remember the hundred emails from companies whe...,0,1,0,0,0,0
3,Today my pop-pop told me I was not “forced” to...,1,0,0,0,0,0
4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1,0,0,0,0,0
...,...,...,...,...,...,...,...
862,yo @claires do yall do hysterectomies?,1,0,0,0,0,1
863,@JacobWohlReport Do I need to aquire a wife be...,1,0,0,0,0,1
864,I get a lot of boy who cried wolf vibes from t...,0,1,0,0,0,0
865,Update: holding hands with your mom and walkin...,1,0,0,0,0,0


In [ ]:
dfExtra = pd.DataFrame({
    "tweet":[
             "Good times, fun times here in Grand old Britannia.",
             "1st Tweet: The Central Division is far from over, @gordonbeckham for a gold glove?  Rios 3-4 tonight @Twins.",
             "Always nice to be forgotten about",
             "Reason 15062017 to get my capsules 🤗Who needs a scarf when your hair is this thick and long?? 😅 #hairgoals #coldweather",
             "love getting a tour of the north west on rail replacement buses 😶",
             "was asked if i wanted my drink in a pint glass or a “lady glass” 🙄 dissapointed as the glass wasnt pink, sparkly or playing the songs from mamma mia",
             "Apparently I lost weight over break? I guess that means I just need to eat more Christmas cookies year round 🤷🏼‍♀️",
             "Happy Bwexit Day #fuckit",
             "Tune in to see old refs throw flags on the ground. There is a football game mixed in between as well.",
             "Still, #Mayweather or #Mcgregor ? PLACE YOUR BETS 🎲💥… https://instagram.com/p/BYRgE1eB6Yc/",
             "so ice cream's supposed to be a drink, right?",
             "some old man told me at work that he knows im montanan cause i say “yup”.  you heard it here first, you can only say yup if youre from montana",
             "Walk from Iowa to Nebraska only took 18 minutes! #jacks",
             "WE INVITE YOU TO BECOME A PART OF SPAIN https://spainisapartofyou.com/gastronomic?v=3&o=fb&r=11238",
             "Fun Fact!: Tails loves cock and ball torture! #themoreyouknow #funfact #sonic",
             "i can’t find my earbuds anywhere. please kill me",
             "love that I finally left my house early to get to uni well before I need to until I realise I forgot my bus pass and locker key :)))",
             "Just earned 5 bounts for being fit and healthy. Start earning now #bounts #JoinThePros",
             "Do I get a girl scout badge for getting punched in the tit by some raging dude wearing sunglasses in the pit of a basement hip hop show?",
             "I love nfl ties!",
             "No offense going to ACE Midwest and meeting Tom Holland would actually suck",
             "i curse the fact that that shit meme makes me laugh everytime",
             "well Americans , prepare to downgrade your house again, soon you will be living in a box .",
             "64: Pikachu Melee: Pikachu Brawl: Pikachu PM: Pikachu S4: Pikachu Ult: Pikachu Oh and did I mention that I like Pikachu?",
             "@peterfacinelli Kids and horse poo..nothing better!",
             "Are we not even eligible to know what happened to this proposed bill? Is this kept aside due to external pressures from private broadcasters?? So at the end people are fooled to take out their time to respond to feedback.",
             "Spooky season is sure off to a spooky start 🚨",
             "You might need a drop of tonic with that 😉🍸",
             "I'm doing the unthinkable and using Python, it's a sad day for Java, I know, but really Python - elif? Was writing elseif too much fuss?",
             "I like to think #Trump actually thinks they are called #hamberders. #McDonaldTrump",
             "If you vote tomorrow I'll buy you an ice cream!",
             "And my folks said I would never amount to anything...",
             "2 years with Mercedes then off to finish career at Ferrari? Tick all the boxes??",
             "just do it in some parking lot somewhere, we all want to hear about what a great job you are doing while we wait to get paid",
             "I'm not religious enough lol",
             "Throat clearing isn't necessary #social_media https://cmun.it/x4bscz",
             "Can the two imbeciles @BorisJohnson & @Jeremy_Hunt please just stop with all the hardline talk & act like proper adults for just a few hours? All your brexit rhetoric is tanking the pound and costing me money on Kickstarters that end today. 1/2",
             "I just went downstairs to get a drink and my mum comes into the kitchen to inform me that “if I tell her @GlamandGore is coming back once more she will not hesitate to disown me”. I guess maybe I’m a little excited? 😂",
             ],
    "sarcasm":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    "irony":[1,0,1,1,1,1,1,1,0,0,0,0,0,0,1,0,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
    "satire":[0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0],
    "understatement":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,1,1],
    "overstatement":[0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    "rhetorical_question":[0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0]
})

df = df.append(dfExtra, ignore_index=True)

In [ ]:
aug = naw.RandomWordAug(action="swap")

def augmentMinorityClass(fichero, veces):
  rango = fichero.shape[0]
  for x in range(veces):
    for i in range(rango):
      if fichero.iloc[i][1]==1: #Clase minoritaria
        nueva_fila = {'tweet':aug.augment(fichero.iloc[i][0]), 'sarcasm':fichero.iloc[i][1], 'irony':fichero.iloc[i][2], 'satire':fichero.iloc[i][3], 'understatement':fichero.iloc[i][4], 'overstatement':fichero.iloc[i][5], 'rhetorical_question':fichero.iloc[i][6]}
        fichero = fichero.append(nueva_fila, ignore_index=True)
  return fichero

In [ ]:
df = augmentMinorityClass(df, 1)

In [ ]:
def crearinputs(text,tokenizer):
   sentences = text
   input_ids = []# For every sentence...
   for sent in sentences:

         encoded_sent = tokenizer.encode(sent,add_special_tokens = True)
         input_ids.append(encoded_sent)# Print sentence 0, now as a list of IDs.
   #print('Max sentence length: ', max([len(sen) for sen in input_ids]))
   #print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)
   #print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_tokid))# Pad our input tokens with value 0.
   input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
   #print('\InputIDS Done.')
   return (input_ids)

In [ ]:
def crearmask(input_ids):
   # Create attention masks
   attention_masks = []# For each sentence...
   for sent in input_ids:
       att_mask = [int(tokid > 0) for tokid in sent]
       attention_masks.append(att_mask)
   #print('\Attention masks Done.')
   return(attention_masks)

In [ ]:
def crearDataLoad(df,dfv,tokenizer):
 dft=dfv  # usamos el del validacion para que nos salga los resultados y no tener que cambiar mucho codigo
 #sentencias=df['tweet']
 sentencias=df['tweet'].str.lower()  # para modelos uncased
 labels=df[etiqueta]
 #test=dfv['tweet']
 test=dft['tweet'].str.lower() # para modelos uncased   ### el fichero de validacion es el que se usa para obtener el resultado final
 ls=dft[etiqueta]
 #validation=dfv['tweet']
 validation=dfv['tweet'].str.lower()  # para modelos uncased  # el fichero que hemos llamado test es usado en la red neuronal
 validation_labels=dfv[etiqueta]
 train=sentencias
 train_labels=labels
 #print("tamaño del train")
 #print(len(train))
 #print("tamaño de la validacion")
 #print(len(validation))
 #print("tamaño del test")
 #print(len(test))
 #print('Loading tokenizer...')
 train_inputs = crearinputs (train,tokenizer)
 #train_inputs, train_labels = balancea (train_inputs, train_labels)
 train_masks= crearmask (train_inputs)
 test_inputs = crearinputs (test,tokenizer)
 test_masks= crearmask (test_inputs)
 validation_inputs = crearinputs (validation,tokenizer)
 validation_masks= crearmask (validation_inputs)
 #print ("distribucion TRAINING antes del resambplin {}".format(Counter(labels)))
 #print ("distribucion TRAINING despues del resambplin {}".format(Counter(train_labels)))
 #print('numero de sentencias TRAINING despues del resampling: ',train_inputs.shape[0])
 #print ("Distribution del test {}".format(Counter(test_labels)))
 #print('numero de sentencias del test: ',test_inputs.shape[0])
 #print ("Distribution del validacion {}".format(Counter(validation_labels)))
 #print('numero de sentencias del validacion: ',validation_inputs.shape[0])
 train_inputs = torch.tensor(train_inputs)
 validation_inputs = torch.tensor(validation_inputs)
 train_labels = torch.tensor(train_labels.values)
 #train_labels = torch.tensor(train_labels)
 validation_labels = torch.tensor(validation_labels.values)
 #validation_labels = torch.tensor(validation_labels)
 train_masks = torch.tensor(train_masks)
 validation_masks = torch.tensor(validation_masks)
 #test_masks= torch.tensor(test_masks)
 #test_inputs= torch.tensor(test_inputs)
 #test_labels= torch.tensor(test_labels.values)
 #test_labels= torch.tensor(test_labels)
 from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler# The DataLoader needs to know our batch size for training, so we specify it
 batch_size= 32
 train_data = TensorDataset(train_inputs, train_masks, train_labels)
 train_sampler = SequentialSampler(train_data)
 train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)# Create the DataLoader for our validation set.
 validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
 validation_sampler = SequentialSampler(validation_data)
 validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)
 #test_data = TensorDataset(test_inputs, test_masks, test_labels)
 #test_sampler = SequentialSampler(test_data)
 #test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)# Create the DataLoader for our validation set.
 return train_dataloader,validation_dataloader #,test_dataloader


In [ ]:
def training_model(train_dataloader,validation_dataloader,model,tokenizer):

 model.cuda()
 from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler# The DataLoader needs to know our batch size for training, so we specify it
 batch_size= 32
 # Note: AdamW is a class from the huggingface library (as opposed to pytorch)
 # I believe the 'W' stands for 'Weight Decay fix"
 optimizer = AdamW(model.parameters(),
   lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
   eps = 1e-8 # args.adam_epsilon - default is 1e-8.
   )
 from transformers import get_linear_schedule_with_warmup# Number of training epochs (authors recommend between 2 and 4)
 epochs = 5
 # Total number of training steps is number of batches * number of epochs.
 total_steps = len(train_dataloader) * epochs# Create the learning rate scheduler.
 #print ('total steps', total_steps)
 scheduler = get_linear_schedule_with_warmup(optimizer,
      num_warmup_steps = 0, # Default value in run_glue.py
      num_training_steps = total_steps)

 import numpy as np# Function to calculate the accuracy of our predictions vs labels

 def flat_accuracy(preds, labels):
   pred_flat = np.argmax(preds, axis=1).flatten()
   labels_flat = labels.flatten()
   return np.sum(pred_flat == labels_flat) / len(labels_flat)

 import time
 import datetime
 def format_time(elapsed):
    # Takes a time in seconds and returns a string hh:mm:ss
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


 import random# This training code is based on the `run_glue.py` script here:
 # https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
 # Set the seed value all over the place to make this reproducible.
 seed_val = 42
 random.seed(seed_val)
 np.random.seed(seed_val)
 torch.manual_seed(seed_val)
 torch.cuda.manual_seed_all(seed_val)# Store the average loss after each epoch so we can plot them.

 loss_values = []# For each epoch...
 acc_values =  []

 def predictions_labels(preds,labels):
    pred = np.argmax(preds,axis=1).flatten()
    label = labels.flatten()
    return pred,label



 for epoch_i in range(0, epochs):

   # ========================================
   # Training
   # ========================================


   print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
   print('Training...') # Measure how long the training epoch takes.
   t0 = time.time() # Reset the total loss for this epoch.
   total_loss = 0 # Put the model into training mode. Don't be mislead--the call to
   total_train_acc = 0
   model.train() # For each batch of training data...
   for step, batch in enumerate(train_dataloader): # Progress update every 40 batches.
     if step % 10 == 0 and not step == 0:
          elapsed = format_time(time.time() - t0)
          print(' Batch {:>5,} of {:>5,}. Elapsed: {:}.'.format(step, len(train_dataloader), elapsed)) # Unpack this training batch from our dataloader.
     b_input_ids = batch[0].to(device)
     b_input_mask = batch[1].to(device)
     b_labels = batch[2].to(device) # Always clear any previously calculated gradients before performing a
     model.zero_grad()  # Perform a forward pass (evaluate the model on this training batch).
     outputs = model(b_input_ids,  attention_mask=b_input_mask,  labels=b_labels) #toktype_ids=None,

     loss = outputs[0] # Accumulate the training loss over all of the batches so that we can
     total_loss += loss.item() # Perform a backward pass to calculate the gradients.
     loss.backward() # Clip the norm of the gradients to 1.0.


     torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Update parameters and take a step using the computed gradient.

     optimizer.step() # Update the learning rate.
     scheduler.step() # Calculate the average loss over the training data.
   avg_train_loss = total_loss / len(train_dataloader)
   loss_values.append(avg_train_loss)

   print(" Average training loss: {0:.2f}".format(avg_train_loss))
   print(" Training epoch took: {:}".format(format_time(time.time() - t0)))
   # ========================================
   # (validation)
   # ========================================
   print("Running validation...")
   t0 = time.time() # Put the model in evaluation mode--the dropout layers behave differently
   # during evaluation.
   model.eval() # Tracking variables
   eval_loss, eval_accuracy = 0, 0
   nb_eval_steps, nb_eval_examples = 0, 0 # Evaluate data for one epoch
   for batch in validation_dataloader:
      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask, b_labels = batch
      with torch.no_grad():  # Forward pass, calculate logit predictions.
          outputs = model(b_input_ids,
                  #toktype_ids=None, #
                  attention_mask=b_input_mask)
      logits = outputs[0] # Move logits and labels to CPU
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()
      tmp_eval_accuracy = flat_accuracy(logits, label_ids)
      eval_accuracy += tmp_eval_accuracy # Track the number of batches
      nb_eval_steps += 1 # Report the final accuracy for this validation run.
   print(" Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
   # Tracking variables test
   print(" Validation took: {:}".format(format_time(time.time() - t0)))
 print("")
 print("Training complete!")
 #torch.save(model.state_dict(),'/content/drive/MyDrive/I2C/Modelos/modeloValidacion'+etiqueta)
 print ('modelo grabado')
 return model
###############################################

In [ ]:
def resultados(validation_dataloader, model, tokenizer):

  model.eval()

  # Tracking variables
  predictions , true_labels = [], []
  pred = []
  t_label =[]
  # Predict
  for batch in validation_dataloader:
    # Add batch to GPU , como no tengo lo dejo aquí
    batch = tuple(t.to(device) for t in batch)

    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch

    # Telling the model not to compute or store gradients, saving memory and
    # speeding up prediction
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, #toktype_ids=None, #
                      attention_mask=b_input_mask)

    logits = outputs[0]


    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Store predictions and true labels
    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

    for l in logits:
      # para cada tupla del logits, se selecciona 0 o 1 dependiendo del valor
      # que sea el mayor (argmax)
      pred_labels_i = np.argmax(l).item()
      pred.append(pred_labels_i)

  pred=np.asarray(pred).tolist()
  t_label = [val for sublist in true_labels for val in sublist] # para aplanar la lista de etiquetas
  #print('predicciones',pred)
  #print('t_labels',t_label)
  #print('validation_labels',validation_labels )
  print("RESULTADOS KFOLD validacion cruzada")
  from sklearn.metrics import confusion_matrix
  from sklearn.metrics import classification_report
  print(classification_report(t_label, pred))
  print ("Distribution test {}".format(Counter(t_label)))
  from sklearn.metrics import confusion_matrix
  print(confusion_matrix(t_label, pred))
  from sklearn.metrics import roc_auc_score
  print('AUC ROC:')
  print(roc_auc_score(t_label, pred))
  from sklearn.metrics import f1_score
  result=f1_score(t_label, pred, average='macro',labels=[0,1],pos_label=1,zero_division=0)
  print('f1-score macro:')
  print(result)
  print("****************************************************************")
  return result

**Ejercutar esta celda para validación cruzada⏪**

In [ ]:
##################### programa principal #######################################
##########################################################
##########################################################
##########################################################
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('no hay gpu')
print('Found GPU at: {}'.format(device_name))

#import torch# If there's a GPU available...
if torch.cuda.is_available():  # Tell PyTorch to use the GPU.
 device = torch.device("cuda")
 print('There are %d GPU(s) available.' % torch.cuda.device_count())
 print('We will use the GPU:', torch.cuda.get_device_name(0))# If not...
else:
 print('No GPU available, using the CPU instead.')
 device = torch.device("cpu")




#### preparando validación cruzada #######
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
train_data = pd.DataFrame(df)
y=pd.DataFrame(df[etiqueta])
n=5
kf = StratifiedKFold(n_splits=n, random_state=1, shuffle=True)
results = []
print ("COMENZANDO PROCESO DE VALIDACION CRUZADA")
for train_index, val_index in kf.split(train_data,y):

    print("Entrenando KFoldStratificada ")
    print("TRAIN:", train_index, "TEST:", val_index)
    # splitting Dataframe (dataset not included)
    train_df = train_data.iloc[train_index]
    val_df = train_data.iloc[val_index]
    modelB =  BertForSequenceClassification.from_pretrained('bert-base-uncased')
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    #tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
    #modelB = AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

    train_dataloader,validation_dataloader = crearDataLoad(train_df,val_df,tokenizer)
    model=training_model(train_dataloader,validation_dataloader, modelB,tokenizer) # el modelo se graba en modeloValidacion
  # obteniendo metricas del modelo generado en el paso anterior
    model.eval()
    result= resultados(validation_dataloader, model,tokenizer)
    # append model score
    results.append(result)


print("results",results)
print(f"Mean-Precision: {sum(results) / len(results)}")

Ejecutar esta celda para entrenamiento normal (20% validación)

In [ ]:
train_data = pd.DataFrame(df)

train_df, validation_df = train_test_split(train_data,random_state=2018, test_size=0.2, stratify=train_data[etiqueta])

##################### programa principal #######################################
##########################################################
##########################################################
##########################################################
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('no hay gpu')
print('Found GPU at: {}'.format(device_name))

#import torch# If there's a GPU available...
if torch.cuda.is_available():  # Tell PyTorch to use the GPU.
 device = torch.device("cuda")
 print('There are %d GPU(s) available.' % torch.cuda.device_count())
 print('We will use the GPU:', torch.cuda.get_device_name(0))# If not...
else:
 print('No GPU available, using the CPU instead.')
 device = torch.device("cpu")


modelB =  BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
#modelB = AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

train_dataloader,validation_dataloader = crearDataLoad(train_df, validation_df,tokenizer)
model=training_model(train_dataloader,validation_dataloader, modelB,tokenizer) # el modelo se graba en modeloValidacion
#torch.save(model.state_dict(),'./prueba2')
# obteniendo metricas del modelo generado en el paso anterior
model.eval()
result = resultados(validation_dataloader, model,tokenizer)
# append model score

### para leer el modelo creado habría que poner algo así
###
#model_state_dict = torch.load("/content/drive/MyDrive/ProfNer/modeloEpoch5/profner",  map_location='cpu')
#model = BertForSequenceClassification.from_pretrained(pretrained_model_name_or_path="dccuchile/bert-base-spanish-wwm-cased", state_dict=model_state_dict, cache_dir='./data')


Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla K80


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


======== Epoch 1 / 5 ========
Training...
 Batch    10 of    41. Elapsed: 0:00:12.
 Batch    20 of    41. Elapsed: 0:00:23.
 Batch    30 of    41. Elapsed: 0:00:35.
 Batch    40 of    41. Elapsed: 0:00:46.
 Average training loss: 0.35
 Training epoch took: 0:00:47
Running validation...
 Accuracy: 0.88
 Validation took: 0:00:04
======== Epoch 2 / 5 ========
Training...
 Batch    10 of    41. Elapsed: 0:00:12.
 Batch    20 of    41. Elapsed: 0:00:24.
 Batch    30 of    41. Elapsed: 0:00:35.
 Batch    40 of    41. Elapsed: 0:00:47.
 Average training loss: 0.25
 Training epoch took: 0:00:48
Running validation...
 Accuracy: 0.86
 Validation took: 0:00:04
======== Epoch 3 / 5 ========
Training...
 Batch    10 of    41. Elapsed: 0:00:12.
 Batch    20 of    41. Elapsed: 0:00:24.
 Batch    30 of    41. Elapsed: 0:00:35.
 Batch    40 of    41. Elapsed: 0:00:47.
 Average training loss: 0.16
 Training epoch took: 0:00:48
Running validation...
 Accuracy: 0.84
 Validation took: 0:00:04
======== Epoc

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/I2C/taskB.En.input.csv', sep=',')

#df_test = df_test.rename(columns={"text":"tweet"})

**Cargamos el modelo**

In [ ]:
### para leer el modelo creado habría que poner algo así
###
model_state_dict = torch.load("/content/drive/MyDrive/I2C/Modelos/modeloValidacionrhetorical_question",  map_location='cpu')
model = BertForSequenceClassification.from_pretrained(pretrained_model_name_or_path="bert-base-uncased", state_dict=model_state_dict, cache_dir='./data')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

**Validación del modelo**

In [ ]:
def crearDataLoadTrial(dfv,tokenizer): #Ponemos df = None por defecto para que podamos usar dataloads de solo los datos de validación (o entrenamiento)

  print("UN SOLO ARGUMENTO")
  dft=dfv  # usamos el del validacion para que nos salga los resultados y no tener que cambiar mucho codigo

  #validation=dfv['text']
  validation=dfv['text'].str.lower()  # para modelos uncased  # el fichero que hemos llamado test es usado en la red neuronal
  #validation_labels=dfv['label']

  validation_inputs = crearinputs (validation,tokenizer)
  validation_masks= crearmask (validation_inputs)

  validation_inputs = torch.tensor(validation_inputs)

  #validation_labels = torch.tensor(validation_labels.values)


  validation_masks = torch.tensor(validation_masks)

  from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler# The DataLoader needs to know our batch size for training, so we specify it

  #Colab
  batch_size = 32
  #local
  #batch_size = 15

  validation_data = TensorDataset(validation_inputs, validation_masks)
  validation_sampler = SequentialSampler(validation_data)
  validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

  return validation_dataloader

In [ ]:
def prediccion(test_dataloader, model, tokenizer):

  model.eval()

  # Tracking variables
  predictions , true_labels = [], []
  pred = []
  t_label =[]
  # Predict
  for batch in test_dataloader:
    # Add batch to GPU , como no tengo lo dejo aquí
    print('Entro')
    batch = tuple(t.to(device) for t in batch)

    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask = batch

    # Telling the model not to compute or store gradients, saving memory and
    # speeding up prediction
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, #toktype_ids=None, #
                      attention_mask=b_input_mask) #Esto falla

    logits = outputs[0] #CREO QUE AQUÍ HABRÍA QUE QUITAR EL 0


    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()


    # Store predictions and true labels
    # Store predictions and true labels
    predictions.append(logits)


    for l in logits:
      # para cada tupla del logits, se selecciona 0 o 1 dependiendo del valor
      # que sea el mayor (argmax)
      pred_labels_i = np.argmax(l).item()
      print(pred_labels_i)
      pred.append(pred_labels_i)

  #Si no me equivoco, en pred guardamos las predicciones hechas por el modelo
  pred=np.asarray(pred).tolist()
  print('predicciones',pred)
  #print('t_labels',t_label)
  #print('validation_labels',validation_labels )

  return pred

In [ ]:
device = torch.device("cpu")
#Probamos la solucion que nos han dicho en stackOverflow
model.to(device)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

validation_dataloader = crearDataLoadTrial(df_test,tokenizer)
# obteniendo metricas del modelo generado en el paso anterior
model.eval()
result= prediccion(validation_dataloader, model,tokenizer)


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

UN SOLO ARGUMENTO
Entro
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
1
0
0
1
1
0
0
0
0
0
0
0
0
0
Entro
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
Entro
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
Entro
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
Entro
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
Entro
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
1
1
0
0
1
1
0
1
0
0
0
0
0
0
Entro
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
Entro
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
Entro
0
0
0
0
1
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
Entro
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Entro
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
Entro
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
Entro
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
Entro
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
En

In [ ]:
prediccionesSarcasmo = result
print(len(prediccionesSarcasmo))

1400


In [ ]:
prediccionesIronia = result
print(len(prediccionesIronia))

1400


In [ ]:
prediccionesSatira = result
print(len(prediccionesSatira))

1400


In [ ]:
prediccionesUnderstatement = result
print(len(prediccionesUnderstatement))

1400


In [ ]:
prediccionesOverstatement = result
print(len(prediccionesOverstatement))

1400


In [ ]:
arr2d = np.zeros((1400,6))

for i in range(arr2d.shape[0]):
  arr2d[i][0]=prediccionesSarcasmo[i]
  arr2d[i][1]=prediccionesIronia[i]
  arr2d[i][2]=prediccionesSatira[i]
  arr2d[i][3]=prediccionesUnderstatement[i]
  arr2d[i][4]=prediccionesOverstatement[i]


In [ ]:
arr2d

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])

In [ ]:
np.savetxt("/content/drive/MyDrive/I2C/entrega3.txt",arr2d, delimiter=",", header= "sarcasm,irony,satire,understatement,overstatement,rhetorical_question", fmt='%1.0f')

In [ ]:
np.savetxt("/content/drive/MyDrive/I2C/entrega31.txt",result, delimiter=",", header= "rhetorical_question", fmt='%1.0f')